In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import wavfile
import os.path
import IPython.display
import seaborn as sns
import librosa
import librosa.display
import soundfile

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPool1D, Flatten, BatchNormalization, ReLU
from keras import optimizers
from sklearn.preprocessing import StandardScaler

import warnings

In [2]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import re
import os
import wave
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from IPython.display import Audio, display
import librosa as lib
import librosa.display

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split

TensorFlow version: 2.12.0


In [5]:

PATH = "/content/drive/MyDrive/Colab-Notebooks/Crema"
AUDIO_PATH = "/content/drive/MyDrive/Colab-Notebooks/Crema/1001_DFA_ANG_XX.wav"

In [6]:
def zeroCrossingRate(audio):
  return lib.feature.zero_crossing_rate(audio)

In [7]:
def energy(audio):
  # using a spectrogram will give a more accurate representation
  # of energy over time because its frames can be windowed
  S, phase = lib.magphase(lib.stft(audio))
  return lib.feature.rms(S=S).mean()

In [8]:
def melSpectrogram(audio, sr):
  mel_spectrogram = lib.feature.melspectrogram(y=audio, sr=sr, n_fft=200)
  log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
  return log_mel_spectrogram.mean()

In [9]:
def chromaStft(audio, sr):
  stft = np.abs(librosa.stft(audio))
  return librosa.feature.chroma_stft(S=stft, sr=sr, n_fft=200).mean()

In [10]:
def mfcc(data, sr):
  return librosa.feature.mfcc(y=data, sr=sr, n_fft=200).mean()

In [11]:
def tonnetz(data, sr):
  return librosa.feature.tonnetz(y=data, sr=sr).mean();

In [12]:
def playAudio(audio_file):
  audio = AudioSegment.from_wav(audio_file)
  # Play the audio
  audio.export('temp_audio.wav', format='wav')
  audio_data = open('temp_audio.wav', 'rb').read()
  display(Audio(audio_data))
  # Delete the temporary audio file
  os.remove('temp_audio.wav')

In [13]:
def visualize_waveform(audio, sr):
    plt.figure(figsize=(12, 4))
    plt.plot(audio)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Waveform')
    plt.show()

In [14]:
def loadAndListenToAudiolib(dataset_path, class_name):
  D, Y = [], []
  # defining the regular expression
  audio_files = glob.glob(os.path.join(dataset_path, f"*{class_name}*.wav"))
  print(class_name)
  # play the audio
  '''playAudio(audio_files[0])'''
  # plot the spectrum
  '''audio, sr = lib.load(audio_files[0])'''
  '''visualize_waveform(audio, sr)'''
  # plotWaveform(audio_files[0], class_name)
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    # chroma stft
    cs = chromaStft(audio, sr)
    mfc = mfcc(audio, sr)
    '''ton = tonnetz(audio, sr)'''
    combined_features = np.concatenate(([rms, mel_spec, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    """print(len(combined_features))"""
    '''combined_features = [zcr, rms, mel_spec, cs, mfc, ton]'''
    # print(combined_features)
    '''D.append(combined_features)'''
    D.append(combined_features)
    Y.append(class_name)
  return D, Y

In [15]:
def loadDatalib(PATH):
  D, Y = [], []
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  for cls in classes: 
    d, y = loadAndListenToAudiolib(PATH, cls)
    # D = np.concatenate((D, d))
    D.extend(d)
    Y = np.concatenate((Y, y))
  return D, Y

In [16]:
'''
D, Y = loadDatalib(PATH)
'''

'\nD, Y = loadDatalib(PATH)\n'

In [17]:
import pickle

In [18]:
'''
# Open a file to write the variable to
with open("D.pkl", "wb") as f:

    # Pickle the variable to the file
    pickle.dump(D, f)

# Close the file
f.close()
'''

'\n# Open a file to write the variable to\nwith open("D.pkl", "wb") as f:\n\n    # Pickle the variable to the file\n    pickle.dump(D, f)\n\n# Close the file\nf.close()\n'

In [19]:
'''
# Open a file to write the variable to
with open("Y.pkl", "wb") as f:

    # Pickle the variable to the file
    pickle.dump(Y, f)

# Close the file
f.close()
'''

'\n# Open a file to write the variable to\nwith open("Y.pkl", "wb") as f:\n\n    # Pickle the variable to the file\n    pickle.dump(Y, f)\n\n# Close the file\nf.close()\n'

In [20]:
# Open the file to read the variable from
with open("D.pkl", "rb") as f:

    # Load the variable from the file
    D = pickle.load(f)

# Close the file
f.close()

In [21]:
# Open the file to read the variable from
with open("Y.pkl", "rb") as f:

    # Load the variable from the file
    Y = pickle.load(f)

# Close the file
f.close()

In [22]:
scaler = StandardScaler()
D = scaler.fit_transform(D)

In [23]:
D = np.array(D)
D.shape

(7442, 304)

In [24]:
D

array([[-0.69282938, -1.36252996,  0.69473674, ...,  0.        ,
         0.        ,  0.        ],
       [-0.72519803, -1.31962175,  1.29348184, ...,  0.        ,
         0.        ,  0.        ],
       [-0.56897265, -0.64803653, -0.41303372, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.53791409, -0.16556313,  0.76860439, ...,  0.        ,
         0.        ,  0.        ],
       [-0.29328666,  0.04421738, -1.5261853 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.46041624, -0.39200239, -0.65966204, ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
Y.shape

(7442,)

In [26]:
df = pd.DataFrame(Y)

In [27]:
df.head()

,0
0,SAD
1,SAD
2,SAD
3,SAD
4,SAD


In [28]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(7442, 6)


In [29]:
Y = one_hot_labels

In [30]:
Y.shape

(7442, 6)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(D, Y, test_size=0.05, random_state=69, shuffle=True, stratify=Y)

In [32]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7069, 304)
(373, 304)
(7069, 6)
(373, 6)


In [33]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [34]:
print(X_train.shape, X_test.shape)

(7069, 304, 1) (373, 304, 1)


In [35]:
unicheck = np.unique(y_test, axis=0)

In [36]:
unicheck

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=uint8)

In [37]:
X_train.shape

(7069, 304, 1)

In [49]:
model = Sequential()
model.add(Conv1D(64, 3, input_shape=(X_train.shape[1], 1), kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(MaxPool1D(pool_size=3))

model.add(Conv1D(128, 3, kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(MaxPool1D(pool_size=3))

model.add(Conv1D(256, 3, kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(MaxPool1D(pool_size=3))

model.add(Conv1D(512, 3, kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(MaxPool1D(pool_size=3))

model.add(Flatten())
model.add(Dense(256, kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(Dropout(0.3))

model.add(Dense(128, kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(Dropout(0.3))

model.add(Dense(64, kernel_regularizer='l2'))
model.add(BatchNormalization(axis=1))
model.add(ReLU())
model.add(Dropout(0.3))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(learning_rate=0.00005),
             metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_24 (Conv1D)          (None, 302, 64)           256       
                                                                 
 batch_normalization_39 (Bat  (None, 302, 64)          1208      
 chNormalization)                                                
                                                                 
 re_lu_39 (ReLU)             (None, 302, 64)           0         
                                                                 
 max_pooling1d_23 (MaxPoolin  (None, 100, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 98, 128)           24704     
                                                                 
 batch_normalization_40 (Bat  (None, 98, 128)         

In [53]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=128,
                    validation_data=(X_test, y_test))

Epoch 1/10
56/56 [==============================] - 1s 21ms/step - loss: 2.0773 - accuracy: 0.8929 - val_loss: 4.0903 - val_accuracy: 0.3566
Epoch 2/10
56/56 [==============================] - 1s 20ms/step - loss: 2.0594 - accuracy: 0.8899 - val_loss: 3.8359 - val_accuracy: 0.4182
Epoch 3/10
56/56 [==============================] - 1s 19ms/step - loss: 2.0169 - accuracy: 0.9032 - val_loss: 3.6879 - val_accuracy: 0.4370
Epoch 4/10
56/56 [==============================] - 1s 19ms/step - loss: 2.0221 - accuracy: 0.8915 - val_loss: 3.8456 - val_accuracy: 0.4048
Epoch 5/10
56/56 [==============================] - 1s 18ms/step - loss: 1.9744 - accuracy: 0.9058 - val_loss: 3.9291 - val_accuracy: 0.3673
Epoch 6/10
56/56 [==============================] - 1s 15ms/step - loss: 1.9519 - accuracy: 0.9113 - val_loss: 3.9324 - val_accuracy: 0.4236
Epoch 7/10
56/56 [==============================] - 1s 15ms/step - loss: 1.9410 - accuracy: 0.9054 - val_loss: 3.7555 - val_accuracy: 0.4075
Epoch 8/10
56